### Han, Joy; Luo, Yutong; Sun, Haichao; Zhang, Stephanie; Zhou, Sophie

# Model Formulations

### Variables

We set our variables to be the number of big dogs (x1), number of small dogs (x2) and fixed costs(x3). It is important to note that we set the variable type to be continuous for our ease to calculate the shadow price. Since shadow price is the slope of a line, the variable type should be set to be continuous rather than integer (discrete).

Objective Function
Our objective is to maximize the daily profit. We formulated our objective function by adding up the profits made by big dogs and small dogs and deducting fixed costs from it. The profits from big dogs are calculated by taking the difference of the revenues and variable costs from big dogs (\\$43-\\$5 = \\$38) and the same fashion applies for the calculation of small dogs (\\$28-\\$3=\\$25). The variable costs are mainly from dog food and different dog supplies such as toy for them to play with. The monthly fixed costs are calculated by summing up the rent & utility costs, insurance, labor and license costs. In order to obtain the daily fixed costs, we divide the monthly fixed costs by 30 days. Therefore, our objective function is:


                                            Maximum. 38x1 + 25x2 - x3


### Constraints

We had two major constraints and another constraint for ease of calculation. The lot size for PUPTOWN GIRLS is only 5000 square feet which sets the limit for the space occupied by both big and small dogs. A big dog will occupy approximately 80 square feet while a small dog will occupy approximately 60 square feet. Therefore, our first constraint is to add up the space occupied by both big and small dogs to be less than or equal to the lot size:

                                                  80x1 + 60x2 <= 5000

The second constraint is related to the number of employees. There are only 10 employees at PUPTOWN GIRLS and every employee could take care of 5 big dogs or 10 small dogs. We transform the coefficients to be the number of employees per big or small dog. Therefore, our second constraint is:

                                                  0.2x1 + 0.1x2 <= 10

Our third constraint is just to put the value of fixed cost calculated before, which is:
    
                                                    x3 = 863.83

Our final model is:

                                             Max. 38x1 + 25x2 - x3
                                            s.t. 80x1 + 60x2 <= 5000
                                              0.2x1 + 0.1x2  <= 10
                                                    x3 = 863.83

The code will be as follows:

In [1]:
import pandas as pd
from gurobipy import *
import openpyxl

In [2]:
# reading the inputs from Excel workbook
wb = openpyxl.load_workbook("Optimization_modeling_dog_dacare.xlsx")
main = wb['Main']

In [3]:
# read objevtive
obj = main.cell(column=2, row=1).value
if obj =='max':
    obj = -1
elif obj =='min':
    obj = 1
else:
    raise NameError('Objective cannot be recognized')
# read obj coefficient
obj_coeff = []
for c in range(2,10000):
    v = main.cell(column=c, row=3).value
    if v is None:
        break
    else:
        obj_coeff.append(v)
n_variables = len(obj_coeff)

#read variable properties
variable_name = []
for c in range(n_variables):
    v = main.cell(column=c+2, row=2).value
    if v is None:
        variable_name.append('x'+str(c+1))
    elif type(v) != str:
        variable_name.append(str(v))
    else:
        variable_name.append(v)
        
variable_type = []
for c in range(n_variables):
    v = main.cell(column=c+2, row=4).value
    if v is None:
        t = 'C' 
    elif v == 'cont':
        t = 'C'
    elif v == 'int':
        t = 'I'
    elif v == 'bin':
        t = 'B'
    else:
        raise NameError('Variable type cannot be recognized')
    variable_type.append(t)
    
variable_lb = []
for c in range(n_variables):
    v = main.cell(column=c+2, row=5).value
    if v is None:
        t = 0
    elif v == '"+inf"':
        raise NameError('Variable lower bound cannot be +inf')
    elif v == '"-inf"':
        t = -GRB.INFINITY
    elif type(v) in [float, int]:
        t = v
    else:
        raise NameError('Variable lower bound cannot be recognized')
    variable_lb.append(t)

variable_ub = []
for c in range(n_variables):
    v = main.cell(column=c+2, row=6).value
    if v is None:
        t = GRB.INFINITY
    elif v == '"+inf"':
        t = GRB.INFINITY
    elif v == '"-inf"':
        raise NameError('Variable upper bound cannot be -inf')
    elif type(v) in [float, int]:
        t = v
    else:
        raise NameError('Variable upper bound cannot be recognized')
    variable_ub.append(t)
#print(obj_coeff)

In [4]:
# read constraints
## find constraint column index
for c in range(1, n_variables+100):
    v = main.cell(column=c, row=8).value
    if v == 'constraint type':
        const_typ_cind = c
        break

if const_typ_cind-2 != n_variables:
    raise NameError('Number of variables does not match the constraints')
    
## find last constraint row index
for r in range(9, 100000):
    v = main.cell(column=const_typ_cind, row=r).value
    if v is None:
        break
    elif v not in ["<=", ">=", "="]:
        raise NameError("Wrong constraint type")
    else:
        last_r_consraints=r

## read constraints as dataframe
for r in range(9, last_r_consraints+1):
    v = main.cell(column=const_typ_cind+2, row=r).value
    if v is None:
        main.cell(column=const_typ_cind+2, row=r).value = "c"+str(r-8)
    elif type(v) != str:
        main.cell(column=const_typ_cind+2, row=r).value = str(v)
    else:
        1==1        
values = []
cons_df = pd.DataFrame()
for r in range(9, last_r_consraints+1):
    r_values = []
    for c in range(2, const_typ_cind+3):
        r_values.append(main.cell(column=c, row=r).value)
    values.append(r_values)
    cons_df = pd.DataFrame(values)
    cons_df.fillna(0, inplace=True)
    cons_df.rename(columns={const_typ_cind-2:'constraint type', 
                            const_typ_cind-1:'RHS values', 
                            const_typ_cind:'constraint name'}, inplace=True)

In [5]:
# setup model
m = Model()

Set parameter Username
Academic license - for non-commercial use only - expires 2023-09-02


In [6]:
# add variables
x=m.addVars(n_variables)
# set types, lb, ub of variables
for i in range(n_variables):
    x[i].setAttr('VarNAME', variable_name[i])
    x[i].setAttr('vType', variable_type[i])
    x[i].setAttr('lb', variable_lb[i])
    x[i].setAttr('ub', variable_ub[i])

In [7]:
# set objective
objective = quicksum(obj_coeff[i] * x[i] for i in range(n_variables))
m.setObjective(objective, obj)

In [8]:
# add constraints
if cons_df.shape[0]:
    ## add le constraints
    le_const = cons_df[cons_df['constraint type'] == '<=']
    le_const.reset_index(inplace=True, drop=True)
    for i in range(le_const.shape[0]):
        m.addConstr(quicksum(le_const.iloc[i,j]* x[j] for j in range(n_variables)) <= le_const['RHS values'][i], 
                    name = le_const['constraint name'][i])
    ## add ge constraints
    ge_const = cons_df[cons_df['constraint type'] == '>=']
    ge_const.reset_index(inplace=True, drop=True)
    for i in range(ge_const.shape[0]):
        m.addConstr(quicksum(ge_const.iloc[i,j]* x[j] for j in range(n_variables)) >= ge_const['RHS values'][i], 
                    name = ge_const['constraint name'][i])
    ## add eq constraints
    eq_const = cons_df[cons_df['constraint type'] == '=']
    eq_const.reset_index(inplace=True, drop=True)
    for i in range(eq_const.shape[0]):
        m.addConstr(quicksum(eq_const.iloc[i,j]* x[j] for j in range(n_variables)) == eq_const['RHS values'][i], 
                    name = eq_const['constraint name'][i])

In [9]:
## run the model
m.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads
Optimize a model with 3 rows, 3 columns and 5 nonzeros
Model fingerprint: 0x44f826ad
Coefficient statistics:
  Matrix range     [1e-01, 8e+01]
  Objective range  [1e+00, 4e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 5e+03]
Presolve removed 1 rows and 1 columns
Presolve time: 0.00s
Presolved: 2 rows, 2 columns, 4 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.5111700e+03   8.979600e+01   0.000000e+00      0s
       2    1.3361700e+03   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.01 seconds (0.00 work units)
Optimal objective  1.336170000e+03


In [10]:
# print optimal solution
m.printAttr('X')


    Variable            X 
-------------------------
      BigDog           25 
    SmallDog           50 
   FixedCost       863.83 


# Results

Finally, after fitting the spreadsheet in Gurobi, the optimal function gives the solution of 25 big dogs and 50 small dogs, which gives the revenue of \\$2,475 (\\$28 * 50 + \\$43 * 25) per day and maximum profit of \\$1,336.17 per day.

We have also done sensitivity analysis. According to the sensitivity analysis, the shadow price for the size of the lot is 0.3 and the shadow price for employees within the range 9 to 12 is 69.99.

# Sensitivity Analysis

To conclude, we have delivered an optimization model for PUPTOWN GIRLS to maximize the profit. The model is fitted in Gurobi and has delivered results for variable optimization. The optimized profit is \\$1,336.17 per day.

This profit is substantial for the business operation and the owner’s family. Moreover, through our shadow price analysis, we have made some suggestions for PUPTOWN GIRLS to increase their profit. One of the constraints is the size of the play area for dogs. We do not recommend PUPTOWN GIRLS to increase their size to enable more customers since the shadow price is only 0.3. So, when increasing the size by 1 square feet, profit only increases by \\$ 0.30. Given the rent in Vancouver, this is economically inefficient. The other constraint is the number of employees. According to the sensitivity analysis, we recommend PUPTOWN GIRLS to hire additional employees to achieve higher profits. Adding one employee will increase the profit by \\$ 70, and the employee’s daily wage is only \\$66.67. So we suggest PUPTOWN GIRLS to increase the number of employees by two more. 

In [11]:
# sensitivity analysis
for v in m.getVars():
    print("For Variable " + v.VarName+ ":")
    print("Minimum value coefficient can take before the optimal decision changes "  + "is " + str(v.SAObjLow))
    print("Maximum value coefficient can take before the optimal decision changes "  + "is " + str(v.SAObjUp))

For Variable BigDog:
Minimum value coefficient can take before the optimal decision changes is 33.333333333333336
Maximum value coefficient can take before the optimal decision changes is 50.00000000000001
For Variable SmallDog:
Minimum value coefficient can take before the optimal decision changes is 19.0
Maximum value coefficient can take before the optimal decision changes is 28.5
For Variable FixedCost:
Minimum value coefficient can take before the optimal decision changes is -inf
Maximum value coefficient can take before the optimal decision changes is inf


In [12]:
for c in m.getConstrs():
    print("For constraint " + c.ConstrName+ ":")
    print("Shawdow Price is " + str(c.pi))
    print("Minimum value RHS can take before the shadow price changes "  + "is " + str(c.SARHSLow))
    print("Maximum value RHS can take before the shadow price changes "  + "is " + str(c.SARHSUp))

For constraint c1:
Shawdow Price is 0.3000000000000001
Minimum value RHS can take before the shadow price changes is 4000.0
Maximum value RHS can take before the shadow price changes is 6000.0
For constraint c2:
Shawdow Price is 69.99999999999996
Minimum value RHS can take before the shadow price changes is 8.333333333333334
Maximum value RHS can take before the shadow price changes is 12.500000000000002
For constraint c3:
Shawdow Price is -1.0
Minimum value RHS can take before the shadow price changes is 0.0
Maximum value RHS can take before the shadow price changes is inf


# Discussion

According to our recommendation, we provided some implementation methods to PUPTOWN GIRLS to reach their optimized number of dogs. If the number of dogs has not reached the optimal level, PUPTOWN GIRLS can spend efforts on advertisements within the community. On the other hand, PUPTOWN GIRLS can make the announcement, as they have reached the daily full capacity, to assure experience and safety of dog stay, thus achieving their optimal value.

Finally, from this case study we not only created this model and reviewed what we learned in class. We also acquired the process of field study, learned about the pet service industry, strengthened problem solving skills and methods to complete a case study, and worked as a team efficiently.
